In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.append("../..")

In [6]:
from json import dumps, loads
from time import time
from typing import Any, NamedTuple

import chardet
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import py_stringmatching as sm
import tensorflow as tf
from sklearn.linear_model import LogisticRegression as skLogisticRegression
from sklearn.metrics import (classification_report, f1_score, precision_score,
                             recall_score)
from sklearn.model_selection import cross_validate, train_test_split
from tensorflow.losses import sigmoid_cross_entropy
from tensorflow.python.ops.parallel_for import jacobian

from mlsql.archiver import Record
from mlsql.experiments.duti import create_gradient_model, create_dataset, create_influence_model, DogCatConfig
from mlsql.tensorflow.problem import LogisticRegression as tfLogisticRegression
from mlsql.tensorflow.utils import EarlyStopTrainer

from tqdm import tnrange

# Single Experiment

In [5]:
seed = 1558545695

In [23]:
corrupt_type = DogCatConfig.Duti
query = "SELECT COUNT(*) WHERE clf() = 1"
EXPERIMENT_NAME = "DUTI-Influence-Retrain"
EXPERIMENT_DESC = {
    "seed": seed,
    "query": query,
    "dataset": "duti",
    "corrupt_type": str(corrupt_type),
}

In [24]:
(Xtrain, ytrain), (Xtest, ytest), ycrptd, sel_crpt = create_dataset(seed, corrupt_type)

In [25]:
Xquery = Xtest

In [26]:
def query_loss_func(Xquery, theta):
    nquery = Xquery.shape[0].value
    sum = tf.reduce_sum(tf.sigmoid(Xquery @ theta))
    return 1 / nquery * tf.norm(ytest.sum() - sum)
    
(theta, delta, logistic_objective), train_logistic_op, influence = create_influence_model(Xtrain, Xquery, ycrptd, query_loss_func)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [27]:
K = int(sel_crpt.sum() * 1.5)
deletions = []

clf = skLogisticRegression(solver="lbfgs", max_iter=500, random_state=seed).fit(Xtrain, ycrptd)
clff1 = f1_score(ytest, clf.predict(Xtest))
clfrecall = f1_score(ytest, clf.predict(Xtest))
clfprecision = f1_score(ytest, clf.predict(Xtest))

vdeltas = np.empty((K, len(Xtrain)))
deletions = np.empty(K)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for k in tnrange(K):
        for _ in EarlyStopTrainer(sess, logistic_objective, max_iter=5000, tol=1e-8):
            sess.run(train_logistic_op)

        vinfluence, vtheta = sess.run([influence, theta])
        
        i = np.argmax(vinfluence)
        
        sess.run(delta[i].assign([0]))
        vdelta = sess.run(delta)
        
        deletions[k] = i
        vdeltas[k,:] = vdelta.squeeze()

In [35]:
tdelta = (1 - sel_crpt)

In [36]:
print(classification_report(tdelta, vdeltas[-1,:]))

              precision    recall  f1-score   support

           0       0.50      0.75      0.60        36
           1       0.80      0.58      0.67        64

   micro avg       0.64      0.64      0.64       100
   macro avg       0.65      0.66      0.64       100
weighted avg       0.69      0.64      0.65       100



In [37]:
r = Record(EXPERIMENT_NAME)
r.input(EXPERIMENT_DESC)
r.output({"model_perf": [clff1, clfprecision, clfrecall], "deltas": vdeltas.tolist(), "tdelta": tdelta.tolist()})
r.insert()

# Multi Runner

In [40]:
seed = 1558545695
EXPERIMENT_NAME = "DUTI-Influence-Retrain"
query = "SELECT COUNT(*) WHERE clf() = 1"

for corrupt_type in DogCatConfig:
    EXPERIMENT_DESC = {
        "seed": seed,
        "query": query,
        "dataset": "duti",
        "corrupt_type": str(corrupt_type),
    }

    desc = f"[{corrupt_type}]"

    (Xtrain, ytrain), (Xtest, ytest), ycrptd, sel_crpt = create_dataset(seed, corrupt_type)

    user_expectation = ytest.sum()

    def query_loss_func(Xquery, theta):
        nquery = Xquery.shape[0].value
        sum = tf.reduce_sum(tf.sigmoid(Xquery @ theta))
        return 1 / nquery * tf.norm(user_expectation - sum)

    clf = skLogisticRegression(solver="lbfgs", max_iter=500, random_state=seed).fit(Xtrain, ycrptd)
    clff1 = f1_score(ytest, clf.predict(Xtest))
    clfrecall = f1_score(ytest, clf.predict(Xtest))
    clfprecision = f1_score(ytest, clf.predict(Xtest))

    Xquery = Xtest

    (theta, delta, logistic_objective), train_logistic_op, influence = create_influence_model(Xtrain, Xquery, ycrptd, query_loss_func)


    K = int(sel_crpt.sum() * 2)
    vdeltas = np.empty((K, len(Xtrain)))
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for k in tnrange(K, desc=desc):
            for _ in EarlyStopTrainer(sess, logistic_objective, max_iter=5000, tol=1e-8):
                sess.run(train_logistic_op)

            vinfluence, vtheta = sess.run([influence, theta])

            i = np.argmax(vinfluence)

            sess.run(delta[i].assign([0]))
            vdelta = sess.run(delta)

            vdeltas[k,:] = vdelta.squeeze()

    tdelta = (1 - sel_crpt)

    r = Record(EXPERIMENT_NAME)
    r.input(EXPERIMENT_DESC)
    r.output({"model_perf": [clff1, clfprecision, clfrecall], "deltas": vdeltas.tolist(), "tdelta": tdelta.tolist()})
    r.insert()